In [ ]:
import numpy as np
import pandas as pd
# import torch
from autogluon.tabular import TabularPredictor
from tqdm import tqdm

# Load datasets
# train = pd.read_csv('./dataset/train_imputed.csv')
# test = pd.read_csv('./dataset/test_imputed.csv')
train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test.csv')
submission = pd.read_csv('./dataset/sample_solution.csv')

In [ ]:
# print(torch.cuda.is_available())

In [ ]:
# Feature and target columns
feature_cols = list(train.columns[:55])
target_cols = list(train.columns[55:])

In [ ]:
fraction_cols = feature_cols[:5]
property_cols = feature_cols[5:]

In [ ]:
# for col in property_cols:
#     component_i = col.split('_')[0][-1]
#     train[f'{col}_times_fraction_{component_i}'] = train[col] * train[f'Component{component_i}_fraction']
#     test[f'{col}_times_fraction_{component_i}'] = test[col] * test[f'Component{component_i}_fraction']

In [ ]:
feature_cols_new = [col for col in train.columns if col not in target_cols]

In [ ]:
def next_best_blend(df_train, target_cols, blend_property, already_selected=None):
    """
    Returns the next best blend based on correlation with BlendProperty1.
    """
    for i in range(-1, -len(target_cols) - 1, -1):
        idx = np.argsort(df_train[target_cols].corr()[blend_property]).iloc[i]
        if target_cols[idx] not in already_selected:
            return target_cols[idx], already_selected + [target_cols[idx]]

blend_property = 'BlendProperty1'
blend_chain = [blend_property]
already_selected = [blend_property, 'BlendProperty3']
while len(already_selected) < len(target_cols):
    next_blend, already_selected = next_best_blend(train, target_cols, blend_property, already_selected)
    blend_chain.append(next_blend)
    blend_property = next_blend
    # print(already_selected)
    # print(f"Next blend: {next_blend}, already selected: {already_selected}")

blend_chain = blend_chain + ['BlendProperty3']
blend_chain

In [ ]:
# Directory to save models
preset = 'best_quality'

output_dir_base = f'./autogluon_models_{preset}/'

# Train one model per target
# for target in tqdm(blend_chain):
for target in tqdm([f'BlendProperty{i}' for i in [9, 10]]):
    print(f"\n🧠 Training AutoGluon model for: {target}")
    
    print('Number of feature columns:', len(feature_cols_new))
    
    train_data = train[feature_cols_new + [target]]
    predictor = TabularPredictor(
        label=target,
        path=f'{output_dir_base}/{target}',
        problem_type='regression',
        eval_metric='mean_absolute_percentage_error',
        verbosity=4,
        
    ).fit(
        train_data=train_data,
        presets=preset,  # Try 'medium_quality_faster_train' for speed
        # hyperparameters=automm_hyperparameters,
        # time_limit=24*60  # seconds (10 min per target)
        # num_gpus=1,
        # included_model_types=['FT_TRANSFORMER'],
        # ag_args_fit={'num_gpus': 1}
    )

    # Predict on test set
    preds = predictor.predict(test[feature_cols_new])
    submission[target] = preds
    
    np.save(f'model_oof_ag_best_b{target[-1]}.npy', predictor.predict_proba_oof().values)
    

    # test[target] = preds
    # feature_cols_new.append(target)

# Save final submission
# submission.to_csv(f'submission_autogluon_time_fraction_{preset}_b9.csv', index=False)

In [ ]:
predictor.leaderboard()

In [ ]:
# pd.read_csv('submission_autogluon_time_fraction_best_quality.csv')

In [ ]:
for target_num in range(9, 11):
    predictor = TabularPredictor.load(f'./autogluon_models_best_quality/BlendProperty{target_num}')
    np.save(f'./ag_preds/pred_b{target_num}' , predictor.predict(test).values)

In [ ]:
# import pickle
# import shap

map_blend_to_mape = {}

for i in tqdm(range(1, 10 + 1)):
    predictor = TabularPredictor.load(output_dir_base + f'/BlendProperty{i}')
    # y_pred = predictor.predict(train[feature_cols])
    # y_true = train[f'BlendProperty{i}']
    # map_blend_to_mape[i] = abs(predictor.evaluate_predictions(y_true, y_pred)['mean_absolute_percentage_error'])
    print('BlendProperty', i)
    print(predictor.leaderboard().head()[['model', 'score_val']])
# map_blend_to_mape



In [ ]:
predictor.leaderboard()

In [ ]:
# train[feature_cols + [target]]
target = target_cols[i-1]

In [ ]:
for i in range(1, 10 + 1):
    target = target_cols[i-1]
    predictor = TabularPredictor.load(f'./autogluon_models/BlendProperty{i}')
    predictor.feature_importance(train[feature_cols + [target]]).to_csv(f'importances/blend{i}.csv')

In [ ]:
i = 2
target = target_cols[i-1]
predictor = TabularPredictor.load(f'./autogluon_models/BlendProperty{i}')
predictor.feature_importance(train[feature_cols + [target]])

In [ ]:
train['BlendProperty3'].max()

In [ ]:
df = pd.read_csv('submission_autogluon_avg_86.45445.csv')

df['BlendProperty7'] = (df['BlendProperty3'] - 0.02)
df['BlendProperty3'] = df['BlendProperty3'].clip(upper=1.63)

df.to_csv('submission_autogluon_avg_86.45445_7=3.csv', index=False)